In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import numpy as np
import os
import pdfplumber

/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <B3E58761-2785-34C6-A89B-F37110C88A05> /Users/alejandromunoz/Library/Python/3.9/lib/pyt

### 1. Aplicamos una función que trasnforma el texto a OCR y lo almacena en una base de datos

In [2]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image

def extraer_texto_de_pdf_con_ocr(ruta_pdf):
    """
    Extrae el texto de un documento PDF utilizando OCR (Tesseract) y pdf2image.
    :param ruta_pdf: Ruta del archivo PDF.
    :return: Texto extraído del PDF.
    """
    texto_completo = ""

    # Convierte el PDF a una lista de imágenes (una imagen por página)
    paginas = convert_from_path(ruta_pdf)

    # Realiza OCR en cada imagen de página
    for numero_pagina, pagina in enumerate(paginas, start=1):
        print(f"Procesando página {numero_pagina}...")  # Informar sobre el progreso
        # Convierte la imagen a un formato soportado por Tesseract (RGB)
        pagina_rgb = pagina.convert('RGB')
        
        # Realiza OCR en la imagen de la página
        texto = pytesseract.image_to_string(pagina_rgb, lang='spa')  # 'lang' establece el idioma (ej. español)
        texto_completo += f"\n\n--- Página {numero_pagina} ---\n" + texto  # Añade el número de página para referencia

    return texto_completo


In [3]:
def crear_df_de_directorio(directorio):
    """
    Lee todos los archivos PDF en un directorio y crea un DataFrame con tres columnas:
    id, titulo (nombre del archivo), texto (contenido extraído).
    :param directorio: Ruta del directorio que contiene los archivos PDF.
    :return: DataFrame con columnas id, titulo, texto.
    """
    archivos = [f for f in os.listdir(directorio) if f.endswith('.pdf')]
    
    datos = []  # Lista para almacenar los datos de los PDFs
    
    for idx, archivo in enumerate(archivos, 1):
        ruta_pdf = os.path.join(directorio, archivo)
        texto = extraer_texto_de_pdf_con_ocr(ruta_pdf)  # Extraemos el texto del PDF
        
        datos.append({
            'id': idx,
            'titulo': archivo,
            'texto': texto
        })
    
    # Crear un DataFrame con los datos recopilados
    df = pd.DataFrame(datos, columns=['id', 'titulo', 'texto'])
    
    return df

In [4]:
directorio = "/Users/alejandromunoz/Desktop/Proyectos/VK/LLM DOCUMENTOS/libros"
df_pdfs = crear_df_de_directorio(directorio)

Procesando página 1...
Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 1...
Procesando página 1...
Procesando página 2...
Procesando página 1...
Procesando página 2...
Procesando página 1...
Procesando página 2...
Procesando página 1...
Procesando página 1...
Procesando página 2...
Procesando página 3...


### 2. Realizamos una limpieza de los datos

In [5]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

# Función para limpiar el texto
def limpiar_texto(texto):
    texto = re.sub(r'•|–|-|–|—', '', texto)  # Eliminar viñetas comunes y guiones
    texto = re.sub(r'\n\s*\n', '\n', texto)  # Eliminar saltos de línea excesivos
    texto = re.sub(r'\s+', ' ', texto)       # Reducir múltiples espacios a uno solo
    texto = texto.strip()                    # Eliminar espacios en blanco al inicio y al final
    return texto

# Función para preprocesar el texto (en español)
def preprocesar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^a-záéíóúñü ]', '', texto)  # Filtrar caracteres no alfabéticos y tildes
    
    stop_words = set(stopwords.words('spanish'))
    palabras = texto.split()
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words]
    
    lemmatizer = WordNetLemmatizer()  # Nota: Este lematizador es para inglés. Considera spaCy para español.
    palabras_lemmatizadas = [lemmatizer.lemmatize(palabra) for palabra in palabras_filtradas]
    
    texto_procesado = ' '.join(palabras_lemmatizadas)
    
    return texto_procesado

# Aplicar las funciones al DataFrame
df_pdfs['texto'] = df_pdfs['texto'].apply(limpiar_texto)
df_pdfs['texto'] = df_pdfs['texto'].apply(preprocesar_texto)



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alejandromunoz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alejandromunoz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# TEXTO LIMPIO
df_pdfs['texto'].iloc[1]

'página sunarp speristendancde necderas et tn egintras pubsicis zona registral n v sede trujillo códig verificación oficina registral trujillo slicitud n registro persona jurídicas libro sociedades anonimas certificado vigencia servidr suscribe certifica partida electrónica n registr persnas jurídicas oficina registral trujillo cnsta registrad vigente nmbramient favr che castillo sandro raziel identificad cn dni n cuys dat precisan cntinuación denominación razón social chimu agropecuaria sa libro sociedades anonimas asiento do cargo jefe sistemas facultades do i confieren jefe sistemas senor sandro raziel che castillo identificado dni n toda atribuciones refieren numeral celebración otorgamiento suscripción actos negocios jurídicos obligaciones contratos celebrar realizar actos contratos negocios jurídicos relativos disposición gravamen afectación activos negociables negocíables muebles inmuebles así modificar concluir tipo actos contratos negoccios jurídicos contrayendo regulando modi

In [19]:
## SET DE PREGUNTAS: 

pregunta = "¿Qué poder se debe otorgar a un jefe de sistemas?"
#'¿Qué tipo de poderes se le otorgan a Sandro Raziel Che Castillo?'
#'Genera un documento para asignación de poderes para un nuevo jefe de sistemas llamado Carlos Rodolfo Zevallos Barranzuela'
#'Dame recomendaciones para poder crear un nuevo documento para asignación de poderos a un jefe de sistemas'
#"Dame un listado de las principales facultades otorgadas a un jefe de sistemas y luego un detalle de cada facultad listada"
#"¿Qué poder se debe otorgar a un jefe de sistemas?"

### 3. Generamos los embeddings a partir de una pregunta

In [20]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
from pymilvus.exceptions import MilvusException
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd

# Conexión a Milvus
def conectar_milvus(host="localhost", port="19530"):
    connections.connect("default", host=host, port=port)
    print("Conexión exitosa a Base de datos Vectorial")

# Creación de colección
def crear_coleccion(nombre_coleccion, embedding_size):
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=embedding_size),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=4096),
        FieldSchema(name="document_id", dtype=DataType.INT64)
    ]
    schema = CollectionSchema(fields, "Colección para almacenar embeddings, textos y document_id")
    
    try:
        collection = Collection(name=nombre_coleccion, schema=schema)
        print(f"Colección '{nombre_coleccion}' creada con éxito")
    except MilvusException as e:
        print(f"Error al crear la colección: {e}")
        collection = Collection(name=nombre_coleccion)

    index_params = {
        "metric_type": "L2",
        "index_type": "HNSW",
        "params": {"M": 16, "efConstruction": 200}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    print("Índice creado con éxito")

    return collection

# Inserción de embeddings, textos y document_id
def insertar_embeddings_y_textos(collection, chunked_documents, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    embeddings = []
    texts = []
    document_ids = []

    for doc in chunked_documents:
        text = doc['text']
        document_id = doc['document_id']
        embedding = model.encode(text)
        embeddings.append(embedding)
        texts.append(text)
        document_ids.append(document_id)

    embeddings_flat = [embedding.tolist() for embedding in embeddings]
    data = [embeddings_flat, texts, document_ids]
    
    collection.insert(data)
    print(f"Insertados {len(embeddings)} embeddings en la colección '{collection.name}'")

# Búsqueda de embeddings en Milvus
def buscar_embeddings(collection, embedding_query, top_k=5):
    embedding_query_flat = embedding_query.tolist()
    collection.load()
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    try:
        resultados = collection.search([embedding_query_flat], "embedding", search_params, limit=top_k, output_fields=["id", "text", "document_id"])

        chunks_encontrados = []
        for resultado in resultados:
            for hit in resultado:
                chunk_info = {
                    "id": hit.id,
                    "text": hit.entity.get('text'),
                    "document_id": hit.entity.get('document_id'),
                    "distance": hit.distance
                }
                chunks_encontrados.append(chunk_info)
                print(f"ID: {chunk_info['id']}, Distancia: {chunk_info['distance']}, Texto: {chunk_info['text']}, Document ID: {chunk_info['document_id']}")

        return chunks_encontrados
    except MilvusException as e:
        print(f"Error durante la búsqueda: {e}")
        raise

# Obtener embedding
def get_embedding(text, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    return model.encode(text)

# Buscar otro chunk en la lista `chunked_documents` con el mismo document_id y la palabra "PROBLEM"
def buscar_chunk_con_problema(chunked_documents, document_id):
    for chunk in chunked_documents:
        if chunk['document_id'] == document_id and 'PROBLEM' in chunk['text']:
            return chunk
    return None

# Integración con Milvus usando df_pdfs
if __name__ == "__main__":
    conectar_milvus()

    nombre_coleccion = "embedding_collection"
    embedding_size = 384  # Tamaño del embedding del modelo
    collection = crear_coleccion(nombre_coleccion, embedding_size)

    # df_pdfs es el DataFrame con las columnas id, titulo y texto
    df_pdfs = df_pdfs  # Asegúrate de que df_pdfs esté correctamente definido antes

    # Dividir documentos en chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
    chunked_documents = []
    chunk_unique_id = 0

    for doc_id, row in df_pdfs.iterrows():
        document = row['texto']  # Extraemos el texto desde la columna 'texto'
        chunks = text_splitter.split_text(document)
        for chunk_id, chunk in enumerate(chunks):
            chunked_documents.append({
                "id": chunk_unique_id,
                "document_id": doc_id,  # Usamos el id de df_pdfs como document_id
                "text": chunk
            })
            chunk_unique_id += 1

    # Insertar los embeddings en la colección de Milvus
    insertar_embeddings_y_textos(collection, chunked_documents)

    pregunta = pregunta
    embedding_pregunta = get_embedding(pregunta)

    top_3emb = buscar_embeddings(collection, embedding_pregunta, top_k=3)
    
    
    

Conexión exitosa a Base de datos Vectorial
Colección 'embedding_collection' creada con éxito
Índice creado con éxito


/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Insertados 44 embeddings en la colección 'embedding_collection'
ID: 453364036813717771, Distancia: 1.1526457071304321, Texto: página sunarp speristendancde necderas et tn egintras pubsicis zona registral n v sede trujillo códig verificación oficina registral trujillo slicitud n registro persona jurídicas libro sociedades anonimas certificado vigencia servidr suscribe certifica partida electrónica n registr persnas jurídicas oficina registral trujillo cnsta registrad vigente nmbramient favr che castillo sandro raziel identificad cn dni n cuys dat precisan cntinuación denominación razón social chimu agropecuaria sa libro sociedades anonimas asiento do cargo jefe sistemas facultades do i confieren jefe sistemas senor sandro raziel che castillo identificado dni n toda atribuciones refieren numeral celebración otorgamiento suscripción actos negocios jurídicos obligaciones contratos celebrar realizar actos contratos negocios jurídicos relativos disposición gravamen afectación activos negocia

### 4. Análisis de la respuesta

In [9]:
## El Embedding más cercano a la pregunta
top_3emb[0]

{'id': 453364036813717771,
 'text': 'página sunarp speristendancde necderas et tn egintras pubsicis zona registral n v sede trujillo códig verificación oficina registral trujillo slicitud n registro persona jurídicas libro sociedades anonimas certificado vigencia servidr suscribe certifica partida electrónica n registr persnas jurídicas oficina registral trujillo cnsta registrad vigente nmbramient favr che castillo sandro raziel identificad cn dni n cuys dat precisan cntinuación denominación razón social chimu agropecuaria sa libro sociedades anonimas asiento do cargo jefe sistemas facultades do i confieren jefe sistemas senor sandro raziel che castillo identificado dni n toda atribuciones refieren numeral celebración otorgamiento suscripción actos negocios jurídicos obligaciones contratos celebrar realizar actos contratos negocios jurídicos relativos disposición gravamen afectación activos negociables negocíables muebles inmuebles así modificar concluir tipo actos contratos negoccios 

In [10]:
df_pdfs

,id,titulo,texto
0,1,019-2024-EF.pdf,página firmad pr editra peru w mw fecha norma ...
1,2,Vigencia Ing. Sandro Che - Chimu - Jefe de Sis...,página sunarp speristendancde necderas et tn e...
2,3,000047-2024-DP.pdf,página firmad pr editra peru w m fecha norma l...
3,4,324-2024.pdf,página firmad pr editra peru w mw fecha w peru...
4,5,307-2024-EF.pdf,página firmad pr editra peru w elw fecha w per...
5,6,017-2024-CR.pdf,página firmad pr editra peru w elw fecha w per...
6,7,210-2024-PCM.pdf,página firmad pr editra peru w m fecha norma l...
7,8,492-MVES.pdf,página firmad pr editra peru w m fecha norma l...


In [20]:
# Para eliminar la base de datos vectorial
#collection.drop()

### 5. Usamos una IA generativa para responder a las preguntas

In [22]:
# MODELO DE LLM
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

# Instanciar el modelo Ollama
llm = Ollama(model="llama3.1:8b")



prompt_template = """
Quiero que respondas a la pregunta que te hago a partir de la información que te estoy dando como contexto

Contexto: {context}

Además, te estoy entregando el nombre del archivo del cual he extraido ese contexto, para que puedas hacer referencia a este al momento de dar tu respuesta. No tienes que acceder al archivo ya que todo lo que necesitas te lo estoy dando en el contexto.

Nombre del Documento: {titulo}

Pregunta: {pregunta}

Quiero que respondas a mi pregunta de manera clara. Primero me das la respuesta y luego me dices de que documento has sacado la información
 
"""

# Función para obtener el resumen utilizando Ollama
def get_summary_ollama(context_text,titulo, pregunta):
    # Crear el prompt final utilizando el contexto
    final_prompt = prompt_template.format(context=context_text, titulo=titulo, pregunta=pregunta)
    
    # Generar la respuesta utilizando el modelo Ollama
    response = llm.invoke(final_prompt)
    
    return response

# Función para procesar la respuesta de Ollama
def get_answer_ollama(context_text, titulo, pregunta):
    # Obtener la respuesta de Ollama
    summary = get_summary_ollama(context_text,titulo, pregunta)
    return summary


context_text = top_3emb[0]['text']
id_documento = top_3emb[0]['document_id']
titulo_documento = df_pdfs['titulo'].iloc[int(id_documento)]

# Pregunta de ejemplo
pregunta = pregunta

# Obtener y mostrar la respuesta de Ollama
print(pregunta)
print()
respuesta = get_answer_ollama(context_text,titulo_documento, pregunta)
print(respuesta)


¿Qué poder se debe otorgar a un jefe de sistemas?

La respuesta a tu pregunta es:

"El jefe de sistemas debe otorgarse el poder para celebrar contratos, realizar actos jurídicos, contratar servicios, financiar inversiones, reinvertir, refinanciar y capitalizar, así como para comprar, vender, permutar, arrendar y locar bienes inmuebles, en cualquiera de sus modalidades y formas."

He sacado esta información del contexto que me proporcionaste, específicamente de la sección que describe las facultades y atribuciones del jefe de sistemas.


In [ ]:
#####
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

# Instanciar el modelo Ollama
llm = Ollama(model="llama3.1:8b")
prompt_template = """

Utiliza el contexto proporcionado para generar un documento en respuesta al requerimiento indicado. Sigue los pasos a continuación:

Contexto: {context}

Además, te proporciono el nombre del archivo del cual se ha extraído dicho contexto para que lo cites al final de tu respuesta. No es necesario que accedas al archivo, ya que toda la información necesaria está en el contexto que te entrego.

Nombre del Documento: {titulo}

Requerimiento: {pregunta}

Instrucciones:

	1.	Responde de manera clara y precisa a la pregunta formulada.
	2.	Luego, menciona el nombre del documento de donde proviene l
 
"""

# Función para obtener el resumen utilizando Ollama
def get_summary_ollama(context_text,titulo, pregunta):
    # Crear el prompt final utilizando el contexto
    final_prompt = prompt_template.format(context=context_text, titulo=titulo, pregunta=pregunta)
    
    # Generar la respuesta utilizando el modelo Ollama
    response = llm.invoke(final_prompt)
    
    return response

# Función para procesar la respuesta de Ollama
def get_answer_ollama(context_text, titulo, pregunta):
    # Obtener la respuesta de Ollama
    summary = get_summary_ollama(context_text,titulo, pregunta)
    return summary


context_text = top_3emb[0]['text']
id_documento = top_3emb[0]['document_id']
titulo_documento = df_pdfs['titulo'].iloc[int(id_documento)]

# Pregunta de ejemplo
pregunta = pregunta

# Obtener y mostrar la respuesta de Ollama
print(pregunta)
print()
respuesta = get_answer_ollama(context_text,titulo_documento, pregunta)
print(respuesta)
